In [1]:
import numpy as np
import math
from bounding_box_utils.bounding_box_utils import iou, convert_coordinates
from ssd_encoder_decoder.matching_utils import match_bipartite_greedy, match_multi
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [2]:
sess = tf.Session()

K.set_session(sess)

In [3]:
gt = np.load("outputs/predder.npy")

In [4]:
gt1 = gt[1]['predictionsK.eval(t)_1']
gt2 = gt[1]['predictions_2']
gt1_proj = gt[1]['predictions_1_proj_tot']
gt2_proj = gt[1]['predictions_2_proj_tot']

In [5]:
pred_1 = np.load("outputs/predictions_1_14.npy")
pred_1_proj = np.load("outputs/predictions_1_proj_14.npy")
pred_2 = np.load("outputs/predictions_2_14.npy")
pred_2_proj = np.load("outputs/predictions_2_proj_14.npy")

labels:  (5, 4)
y_encoded:  (17292, 4)
similarities:  (5, 17292)
bipartite_matches:  (5,)
similarities after bipartite:  (5, 17292)
matches:  (53,)  -  (53,)
similarities after matches:  (5, 17292)

In [7]:
pred_1 = pred_1_proj[:,:,:18]
gt_1 = gt1_proj[:,:,:18]
gt_2 = gt2_proj[:,:,18:]

In [8]:
filterer = np.where(pred_1[:,:,0]>0.5)
filterer2 = np.where(gt_1[:,:,-1]!=99)

In [9]:
# src = src[filterer]
pred_1 = pred_1[0,:,:]
gt_1 = gt_1[filterer2]
gt_2 = gt_2[filterer2]

In [ ]:
# src = src[0,:,:]
print(pred_1.shape)
print(gt_1.shape)
print(gt_2.shape)

In [10]:
pred_2 = pred_1_proj[:,:,18:]

In [11]:
iou_out = tf.py_func(iou, [tf.convert_to_tensor(gt_1[:,-16:-12]), tf.convert_to_tensor(pred_1[:,-16:-12])], tf.float64)

In [12]:
bipartite_matches = tf.py_func(match_bipartite_greedy, [iou_out], tf.int64)
print(K.eval(bipartite_matches).shape)

(4,)


In [15]:
def finder(tensor, matches):
    return tensor[:,matches,:]
    

In [50]:
t = tf.py_func(finder, [tf.convert_to_tensor(pred_2),bipartite_matches], tf.float32)

In [61]:
all_elems_equal = tf.reduce_all(tf.equal(t, t))

In [62]:
K.eval(all_elems_equal)

True

In [71]:
K.eval(tf.reduce_all(tf.equal(tf.shape(t)[:], tf.shape(t)[:])))

True

In [63]:
tf.shape(t)

<tf.Tensor 'Shape_46:0' shape=(?,) dtype=int32>

In [64]:
tf.shape(t)

<tf.Tensor 'Shape_47:0' shape=(?,) dtype=int32>